## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [1]:
# import libraries
import pandas as pd

In [2]:
ckd = pd.read_csv("chronic_kidney_disease_full.csv")

In [3]:
ckd.head(10)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
5,60.0,90.0,1.015,3.0,0.0,NaN,NaN,notpresent,notpresent,74.0,...,39.0,7800.0,4.4,yes,yes,no,good,yes,no,ckd
6,68.0,70.0,1.010,0.0,0.0,NaN,normal,notpresent,notpresent,100.0,...,36.0,NaN,NaN,no,no,no,good,no,no,ckd
7,24.0,NaN,1.015,2.0,4.0,normal,abnormal,notpresent,notpresent,410.0,...,44.0,6900.0,5.0,no,yes,no,good,yes,no,ckd
8,52.0,100.0,1.015,3.0,0.0,normal,abnormal,present,notpresent,138.0,...,33.0,9600.0,4.0,yes,yes,no,good,no,yes,ckd
9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,...,29.0,12100.0,3.7,yes,yes,no,poor,no,yes,ckd


In [4]:
ckd.dtypes

age      float64
bp       float64
sg       float64
al       float64
su       float64
rbc       object
pc        object
pcc       object
ba        object
bgr      float64
bu       float64
sc       float64
sod      float64
pot      float64
hemo     float64
pcv      float64
wbcc     float64
rbcc     float64
htn       object
dm        object
cad       object
appet     object
pe        object
ane       object
class     object
dtype: object

In [5]:
ckd.groupby('class')['class'].count()

class
ckd       250
notckd    150
Name: class, dtype: int64

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

Answer:
- Compare the other features with `ckd` to see if they have a strong correlation with `ckd`.
- Are there any missing data that could significantly affect the analysis.
- Interaction terms consisting of the variables with 'blood' may be included when constructing a linear model.

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [6]:
ckd.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [7]:
# convert to percentages
ckd.isnull().sum() / 400 * 100

age       2.25
bp        3.00
sg       11.75
al       11.50
su       12.25
rbc      38.00
pc       16.25
pcc       1.00
ba        1.00
bgr      11.00
bu        4.75
sc        4.25
sod      21.75
pot      22.00
hemo     13.00
pcv      17.75
wbcc     26.50
rbcc     32.75
htn       0.50
dm        0.50
cad       0.50
appet     0.25
pe        0.25
ane       0.25
class     0.00
dtype: float64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [8]:
# check total number of null rows
ckd.dropna(axis = 0, how = 'any', inplace = False)['class'].value_counts()

class
notckd    115
ckd        43
Name: count, dtype: int64

Answer:
- If we drop all 158 rows with at least 1 null value, there would be 242 rows left. This means that roughly 40% of the rows will be dropped.
- This could significantly affect the data analysis as the other columns in these dropped rows may contain useful information that could affect the model.
- Additionally, the NaN values could possibly indicate that the medical team has yet to measure the patient's count levels.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer:
1. Input for certain features like `appetite` (`good` vs. `poor`) may be slightly subjective.
2. The definition for `normal` and `abnormal` for `red blood cells` and `pus cells` may not be standardised.
3. Since there are many different staff tending to different patients on the daily, there might be occurrences of human error when inputting patient information.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

Answer: The model will always tell patients that they have CKD even if they do not have CKD.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: Minimising false negatives, which also means maximising sensitivity.

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Answer: It would cause unnecessary distress/misery/anxiety for patients who do not actually have CKD.

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Answer: The model will always tell patients that they DO NOT have CKD even if they do have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

Answer: Minimising false positives, which also means maximising specificity.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

Answer: Patients who actually have CKD will end up undiagnosed and untreated. This poses a serious liabiity for the medical institution where they could even be held legally liable.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [9]:
ckd.head(10)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
5,60.0,90.0,1.015,3.0,0.0,NaN,NaN,notpresent,notpresent,74.0,...,39.0,7800.0,4.4,yes,yes,no,good,yes,no,ckd
6,68.0,70.0,1.010,0.0,0.0,NaN,normal,notpresent,notpresent,100.0,...,36.0,NaN,NaN,no,no,no,good,no,no,ckd
7,24.0,NaN,1.015,2.0,4.0,normal,abnormal,notpresent,notpresent,410.0,...,44.0,6900.0,5.0,no,yes,no,good,yes,no,ckd
8,52.0,100.0,1.015,3.0,0.0,normal,abnormal,present,notpresent,138.0,...,33.0,9600.0,4.0,yes,yes,no,good,no,yes,ckd
9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,...,29.0,12100.0,3.7,yes,yes,no,poor,no,yes,ckd


In [10]:
# create another column where 1 represents patients with ckd and 0 with no ckd
ckd['y'] = [1 if i == 'ckd' else 0 for i in ckd['class']]

In [11]:
# replace pc and pcc columns with 1 and 0
ckd['pc'] = [1 if i == 'abnormal' else 0 for i in ckd['pc']]
ckd['pcc'] = [1 if i == 'present' else 0 for i in ckd['pcc']]

In [12]:
ckd

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class,y
0,48.0,80.0,1.020,1.0,0.0,NaN,0,0,notpresent,121.0,...,7800.0,5.2,yes,yes,no,good,no,no,ckd,1
1,7.0,50.0,1.020,4.0,0.0,NaN,0,0,notpresent,NaN,...,6000.0,NaN,no,no,no,good,no,no,ckd,1
2,62.0,80.0,1.010,2.0,3.0,normal,0,0,notpresent,423.0,...,7500.0,NaN,no,yes,no,poor,no,yes,ckd,1
3,48.0,70.0,1.005,4.0,0.0,normal,1,1,notpresent,117.0,...,6700.0,3.9,yes,no,no,poor,yes,yes,ckd,1
4,51.0,80.0,1.010,2.0,0.0,normal,0,0,notpresent,106.0,...,7300.0,4.6,no,no,no,good,no,no,ckd,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,0,0,notpresent,140.0,...,6700.0,4.9,no,no,no,good,no,no,notckd,0
396,42.0,70.0,1.025,0.0,0.0,normal,0,0,notpresent,75.0,...,7800.0,6.2,no,no,no,good,no,no,notckd,0
397,12.0,80.0,1.020,0.0,0.0,normal,0,0,notpresent,100.0,...,6600.0,5.4,no,no,no,good,no,no,notckd,0
398,17.0,60.0,1.025,0.0,0.0,normal,0,0,notpresent,114.0,...,7200.0,5.9,no,no,no,good,no,no,notckd,0


In [13]:
# create interaction terms
ckd['pc_pcc_interaction'] = ckd['pc'] * ckd['pcc']
ckd['wbcc_rbcc_interaction'] = ckd['wbcc'] * ckd['rbcc']

In [14]:
ckd

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,htn,dm,cad,appet,pe,ane,class,y,pc_pcc_interaction,wbcc_rbcc_interaction
0,48.0,80.0,1.020,1.0,0.0,NaN,0,0,notpresent,121.0,...,yes,yes,no,good,no,no,ckd,1,0,40560.0
1,7.0,50.0,1.020,4.0,0.0,NaN,0,0,notpresent,NaN,...,no,no,no,good,no,no,ckd,1,0,NaN
2,62.0,80.0,1.010,2.0,3.0,normal,0,0,notpresent,423.0,...,no,yes,no,poor,no,yes,ckd,1,0,NaN
3,48.0,70.0,1.005,4.0,0.0,normal,1,1,notpresent,117.0,...,yes,no,no,poor,yes,yes,ckd,1,1,26130.0
4,51.0,80.0,1.010,2.0,0.0,normal,0,0,notpresent,106.0,...,no,no,no,good,no,no,ckd,1,0,33580.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,0,0,notpresent,140.0,...,no,no,no,good,no,no,notckd,0,0,32830.0
396,42.0,70.0,1.025,0.0,0.0,normal,0,0,notpresent,75.0,...,no,no,no,good,no,no,notckd,0,0,48360.0
397,12.0,80.0,1.020,0.0,0.0,normal,0,0,notpresent,100.0,...,no,no,no,good,no,no,notckd,0,0,35640.0
398,17.0,60.0,1.025,0.0,0.0,normal,0,0,notpresent,114.0,...,no,no,no,good,no,no,notckd,0,0,42480.0


In [15]:
# dummify categorical variables
rbc_abnormal = pd.get_dummies(ckd['rbc'])['abnormal']
ba_present = pd.get_dummies(ckd['ba'])['present']
htn_yes = pd.get_dummies(ckd['htn'])['yes']
dm_yes = pd.get_dummies(ckd['dm'])['yes']
cad_yes = pd.get_dummies(ckd['cad'])['yes']
appet_poor = pd.get_dummies(ckd['appet'])['poor']
pe_yes = pd.get_dummies(ckd['pe'])['yes']
ane_yes = pd.get_dummies(ckd['ane'])['yes']

In [16]:
# drop non-numeric columns
numeric = ckd.drop(columns=['rbc','ba','htn','dm','cad','appet','pe','ane','class','y'])

In [17]:
# add dummify columns
dummies = pd.DataFrame([ane_yes, pe_yes, rbc_abnormal, ba_present, htn_yes, dm_yes,
                        cad_yes, appet_poor],
                        index=['ane_yes', 'pe_yes', 'rbc_abnormal',
                               'ba_present', 'htn_yes', 'dm_yes',
                               'cad_yes', 'appet_poor']).T
dummies = dummies.astype(int)

X = numeric.merge(right=dummies, left_index=True, right_index=True)
X

,age,bp,sg,al,su,pc,pcc,bgr,bu,sc,...,pc_pcc_interaction,wbcc_rbcc_interaction,ane_yes,pe_yes,rbc_abnormal,ba_present,htn_yes,dm_yes,cad_yes,appet_poor
0,48.0,80.0,1.020,1.0,0.0,0,0,121.0,36.0,1.2,...,0,40560.0,0,0,0,0,1,1,0,0
1,7.0,50.0,1.020,4.0,0.0,0,0,NaN,18.0,0.8,...,0,NaN,0,0,0,0,0,0,0,0
2,62.0,80.0,1.010,2.0,3.0,0,0,423.0,53.0,1.8,...,0,NaN,1,0,0,0,0,1,0,1
3,48.0,70.0,1.005,4.0,0.0,1,1,117.0,56.0,3.8,...,1,26130.0,1,1,0,0,1,0,0,1
4,51.0,80.0,1.010,2.0,0.0,0,0,106.0,26.0,1.4,...,0,33580.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,0,0,140.0,49.0,0.5,...,0,32830.0,0,0,0,0,0,0,0,0
396,42.0,70.0,1.025,0.0,0.0,0,0,75.0,31.0,1.2,...,0,48360.0,0,0,0,0,0,0,0,0
397,12.0,80.0,1.020,0.0,0.0,0,0,100.0,26.0,0.6,...,0,35640.0,0,0,0,0,0,0,0,0
398,17.0,60.0,1.025,0.0,0.0,0,0,114.0,50.0,1.0,...,0,42480.0,0,0,0,0,0,0,0,0


In [18]:
# import library for L1 and L2 regularisation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [19]:
X.mean()

age                         51.483376
bp                          76.469072
sg                           1.017408
al                           1.016949
su                           0.450142
pc                           0.190000
pcc                          0.105000
bgr                        148.036517
bu                          57.425722
sc                           3.072454
sod                        137.528754
pot                          4.627244
hemo                        12.526437
pcv                         38.884498
wbcc                      8406.122449
rbcc                         4.707435
pc_pcc_interaction           0.082500
wbcc_rbcc_interaction    39260.485075
ane_yes                      0.150000
pe_yes                       0.190000
rbc_abnormal                 0.117500
ba_present                   0.055000
htn_yes                      0.367500
dm_yes                       0.342500
cad_yes                      0.085000
appet_poor                   0.205000
dtype: float

In [20]:
# fill all NaN values with mean
X_train, X_test, y_train, y_test = train_test_split(X.fillna(X.mean()),
                                                    ckd['y'],
                                                    test_size = 0.3, 
                                                    random_state = 42)

In [21]:
parameters = {'C': [0.001, 0.01, 0.1, 1, 10],
              'class_weight': [None, 'balanced'],
              'penalty': ['l1', 'l2']}

In [22]:
import random
random.seed(42)

# test l1 and l2 penalties in GridSearch
lr = LogisticRegression(solver = 'liblinear', 
                        max_iter = 1000,
                        random_state = 42)

gs_results = GridSearchCV(estimator = lr, # specify model to GridSearch
                          param_grid = parameters, # specify the grid of parameters to search
                          scoring = 'recall', # specify recall as the metric to optimise 
                          cv = 5).fit(X_train, y_train) # set 5-fold cross-validation, then fit

In [23]:
gs_results.best_estimator_.get_params()

{'C': 10,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

Based on the results of GridSearch, the best model
- has an inverse regularisation strength of C = 10
- has a balanced class weight (50% ckd and 50% no ckd)
- has the l2 penalty

In [24]:
logit = LogisticRegression(solver = 'liblinear', 
                           max_iter = 1000,
                           C = 10,
                           class_weight = 'balanced',
                           penalty = 'l2',
                           random_state = 42)

In [25]:
logit.fit(X = X_train,
          y = y_train)

LogisticRegression(C=10, class_weight='balanced', max_iter=1000,
                   random_state=42, solver='liblinear')

In [26]:
logit.score(X_train, y_train)

0.9857142857142858

In [27]:
logit.score(X_test, y_test)

0.975

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [28]:
import numpy as np
list(zip(np.exp(logit.coef_[0]),X.columns))

[(1.010470528731073, 'age'),
 (1.1206418451003968, 'bp'),
 (1.1869020724359562, 'sg'),
 (30.335070564982708, 'al'),
 (2.8620252739452825, 'su'),
 (1.9626523176586945, 'pc'),
 (1.0832488516623722, 'pcc'),
 (1.037902349956163, 'bgr'),
 (0.92663397063046, 'bu'),
 (11.823510082987045, 'sc'),
 (1.068883465130749, 'sod'),
 (1.331481644368559, 'pot'),
 (0.18841273164374436, 'hemo'),
 (1.0038932041214972, 'pcv'),
 (0.99961933470643, 'wbcc'),
 (0.35376205369038344, 'rbcc'),
 (1.0676087372458802, 'pc_pcc_interaction'),
 (1.0000738866964134, 'wbcc_rbcc_interaction'),
 (1.6314522325576848, 'ane_yes'),
 (5.530685319819266, 'pe_yes'),
 (2.0763013897083016, 'rbc_abnormal'),
 (1.02363525960652, 'ba_present'),
 (4.217617441907275, 'htn_yes'),
 (4.100872433296188, 'dm_yes'),
 (1.0605295610527083, 'cad_yes'),
 (4.653317798052436, 'appet_poor')]

**Interpretation:** If a patient's albumin `al` increases by one unit, the patient is about 30.3x more likely to have CKD.

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [29]:
list(zip(np.exp(logit.coef_[0]),X.columns))

[(1.010470528731073, 'age'),
 (1.1206418451003968, 'bp'),
 (1.1869020724359562, 'sg'),
 (30.335070564982708, 'al'),
 (2.8620252739452825, 'su'),
 (1.9626523176586945, 'pc'),
 (1.0832488516623722, 'pcc'),
 (1.037902349956163, 'bgr'),
 (0.92663397063046, 'bu'),
 (11.823510082987045, 'sc'),
 (1.068883465130749, 'sod'),
 (1.331481644368559, 'pot'),
 (0.18841273164374436, 'hemo'),
 (1.0038932041214972, 'pcv'),
 (0.99961933470643, 'wbcc'),
 (0.35376205369038344, 'rbcc'),
 (1.0676087372458802, 'pc_pcc_interaction'),
 (1.0000738866964134, 'wbcc_rbcc_interaction'),
 (1.6314522325576848, 'ane_yes'),
 (5.530685319819266, 'pe_yes'),
 (2.0763013897083016, 'rbc_abnormal'),
 (1.02363525960652, 'ba_present'),
 (4.217617441907275, 'htn_yes'),
 (4.100872433296188, 'dm_yes'),
 (1.0605295610527083, 'cad_yes'),
 (4.653317798052436, 'appet_poor')]

**Interpretation:** If a patient tests positive for pedal edema `pe`, the patient is about 5.5x more likely to have CKD.

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer:

1. **Interpretability:** Logistic regression provides easily interpretable results, helping stakeholders understand how individual features influence the predicted outcome.

2. **Efficiency with Binary Classification:** Logistic regression efficiently handles binary classification tasks, making it suitable for real-time applications and scenarios with limited computational resources.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

Answer: Generally, it is better to generate a confusion matrix on test data rather than training data to accurately evaluate how well a model generalizes to new, unseen examples and identify potential issues such as overfitting or bias.

In [30]:
y_preds = logit.predict(X_test)

In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
confusion_matrix(y_test, y_preds)

array([[43,  1],
       [ 2, 74]])

In [33]:
tn, fp, fn, tp = confusion_matrix(y_test, y_preds).ravel()

In [34]:
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

True Negatives: 43
False Positives: 1
False Negatives: 2
True Positives: 74


### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer:
- In this case, optimising sensitivity should be prioritised to minimise false negatives and ensure that potential health issues are not overlooked.
- Specificity is also important, to a lesser degree, as we want to minimise false positives where we want as few people as possible to be diagnosed with CKD without actually having the condition.
- Though sensitivity is more important, we could also look at f1 score which is a combination of sensitivity and specificity.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Answer:
- Unbalanced classes in data science lead to biased models, misleading evaluation metrics, and costly errors, making it challenging to effectively learn from and predict rare events or minority classes.
- In this case, however, a 40/60 split is not a significant imbalance.
- GridSearch also found that the model performs better when the `y` classes are balanced.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Answer: In medical diagnosis, consider a dataset where 95% of patients are healthy (negative class) and only 5% have a rare disease (positive class). If a machine learning model is trained on this imbalanced dataset without proper handling, it may achieve high accuracy by simply predicting all instances as negative. However, this would overlook the minority class of patients with the rare disease, leading to potentially severe consequences such as delayed diagnosis and treatment, ultimately impacting patient outcomes and healthcare quality.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [35]:
ckd_sample = ckd[ckd['class'] == 'ckd'].sample(200_000,            # sample n = 200,000
                                               replace = True,     # sample with replacement
                                               random_state = 42)  # set random state

In [36]:
ckd_combined = pd.concat([ckd, ckd_sample])

In [37]:
ckd_combined

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,htn,dm,cad,appet,pe,ane,class,y,pc_pcc_interaction,wbcc_rbcc_interaction
0,48.0,80.0,1.020,1.0,0.0,NaN,0,0,notpresent,121.0,...,yes,yes,no,good,no,no,ckd,1,0,40560.0
1,7.0,50.0,1.020,4.0,0.0,NaN,0,0,notpresent,NaN,...,no,no,no,good,no,no,ckd,1,0,NaN
2,62.0,80.0,1.010,2.0,3.0,normal,0,0,notpresent,423.0,...,no,yes,no,poor,no,yes,ckd,1,0,NaN
3,48.0,70.0,1.005,4.0,0.0,normal,1,1,notpresent,117.0,...,yes,no,no,poor,yes,yes,ckd,1,1,26130.0
4,51.0,80.0,1.010,2.0,0.0,normal,0,0,notpresent,106.0,...,no,no,no,good,no,no,ckd,1,0,33580.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,65.0,90.0,1.020,2.0,1.0,abnormal,0,0,notpresent,270.0,...,yes,yes,no,poor,no,yes,ckd,1,0,48020.0
6,68.0,70.0,1.010,0.0,0.0,NaN,0,0,notpresent,100.0,...,no,no,no,good,no,no,ckd,1,0,NaN
231,60.0,90.0,NaN,NaN,NaN,NaN,0,0,notpresent,269.0,...,yes,yes,yes,good,yes,no,ckd,1,0,NaN
90,63.0,100.0,1.010,2.0,2.0,normal,0,0,present,280.0,...,yes,no,yes,good,no,no,ckd,1,0,41160.0


In [38]:
ckd_combined.groupby('class')['class'].count() / 200400 * 100

class
ckd       99.92515
notckd     0.07485
Name: class, dtype: float64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [39]:
quant = ckd_combined[['age', 'bp', 'sg', 'al', 'su', 'bgr',
                      'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv',
                      'wbcc_rbcc_interaction', 'y']].copy()

quant.loc[:,'rbc_abnormal_2'] = pd.get_dummies(ckd_combined['rbc'])['abnormal']
quant.loc[:,'ba_present_2'] = pd.get_dummies(ckd_combined['ba'])['present']
quant.loc[:,'htn_yes_2'] = pd.get_dummies(ckd_combined['htn'])['yes']
quant.loc[:,'dm_yes_2'] = pd.get_dummies(ckd_combined['dm'])['yes']
quant.loc[:,'cad_yes_2'] = pd.get_dummies(ckd_combined['cad'])['yes']
quant.loc[:,'appet_poor_2'] = pd.get_dummies(ckd_combined['appet'])['poor']
quant.loc[:,'pe_yes_2'] = pd.get_dummies(ckd_combined['pe'])['yes']
quant.loc[:,'ane_yes_2'] = pd.get_dummies(ckd_combined['ane'])['yes']

In [40]:
quant.head(10)

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,wbcc_rbcc_interaction,y,rbc_abnormal_2,ba_present_2,htn_yes_2,dm_yes_2,cad_yes_2,appet_poor_2,pe_yes_2,ane_yes_2
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,...,40560.0,1,False,False,True,True,False,False,False,False
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,...,NaN,1,False,False,False,False,False,False,False,False
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,...,NaN,1,False,False,False,True,False,True,False,True
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,...,26130.0,1,False,False,True,False,False,True,True,True
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,...,33580.0,1,False,False,False,False,False,False,False,False
5,60.0,90.0,1.015,3.0,0.0,74.0,25.0,1.1,142.0,3.2,...,34320.0,1,False,False,True,True,False,False,True,False
6,68.0,70.0,1.010,0.0,0.0,100.0,54.0,24.0,104.0,4.0,...,NaN,1,False,False,False,False,False,False,False,False
7,24.0,NaN,1.015,2.0,4.0,410.0,31.0,1.1,NaN,NaN,...,34500.0,1,False,False,False,True,False,False,True,False
8,52.0,100.0,1.015,3.0,0.0,138.0,60.0,1.9,NaN,NaN,...,38400.0,1,False,False,True,True,False,False,False,True
9,53.0,90.0,1.020,2.0,0.0,70.0,107.0,7.2,114.0,3.7,...,44770.0,1,True,False,True,True,False,True,False,True


In [41]:
X_train, X_test, y_train, y_test = train_test_split(quant.fillna(quant.mean()).drop(['y'],axis = 1),
                                                    quant['y'],
                                                    test_size = 0.3, 
                                                    random_state = 42)

In [42]:
logit_2 = LogisticRegression(solver = 'liblinear',
                             max_iter = 1000,
                             C = 10,
                             random_state = 42,
                             penalty = 'l2')

In [43]:
logit_2.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [44]:
from sklearn.metrics import classification_report

In [45]:
print(classification_report(y_test,
                            logit_2.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.59      0.74        56
           1       1.00      1.00      1.00     60064

    accuracy                           1.00     60120
   macro avg       1.00      0.79      0.87     60120
weighted avg       1.00      1.00      1.00     60120



Answer:
- Sensitivity = TP/(TP+FN)
- Sensitivity for 1st logistic regression model is 74/76 which is about 93%.
- Sensitivity for 2nd logistic regression model is 100%.
- While it seems that unbalanced classes cause the model to perform better, the issue is that the algorithm is unable to learn the patterns for the minority class as there is insufficient data.
- This can lead to an overestimation of the model's effectiveness where correctly identifying CKD is crucial.
- To mitigate these issues, techniques such as resampling methods (e.g., oversampling, undersampling) and using a combination of appropriate evaluation metrics (e.g., precision, recall, F1-score) may be employed. These approaches help to improve the model's ability to accurately classify instances from both the majority and minority classes.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!